In [63]:
# Скачать файл из Google Drive
! gdown --id 1YsLUVR9bc2WQRIVC0Os1TCyx-yTwQRHq

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1YsLUVR9bc2WQRIVC0Os1TCyx-yTwQRHq
To: /content/train.csv
100% 61.2k/61.2k [00:00<00:00, 4.27MB/s]


In [64]:
# Импорт модулей
import pandas as pd
import numpy as np

In [65]:
# Функция, которая будет искать имя пассажира
def fn(input_string):
    # Проверяем, есть ли в строке скобка '('
    if '(' in input_string:
        # Разделяем строку по символу '(' и берем второй кусок
        second_part = input_string.split('(')[1]
        # Разделяем второй кусок пробелом и берем второй элемент
        result = second_part.split()[0] if len(second_part.split()) > 1 else None
    else:
        # Если скобки нет, то делим по ". "
        second_part = input_string.split('.')[1]
        result = second_part.split()[0]

    return result

### **Считайте датасет из файла**

In [66]:
# Чтение CSV
df = pd.read_csv('/content/train.csv')
#df['first_name'] = df['Name'].str.split(',').str[0]
df['first_name'] = df['Name'].apply(fn)
df[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,first_name
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Owen
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Florence
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Laina
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Lily


### **Выведите основную информацию о датасете: информацию о типах данных, число пропусков, средние значения и т.д.**

In [67]:
# Строки и столбцы
print("Количество строк:", df.shape[0])
print("Количество столбцов:", df.shape[1])

Количество строк: 891
Количество столбцов: 13


In [68]:
print("Типы данных:")
df.dtypes

Типы данных:


,0
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [69]:
# Пропущенные значения
print("Пропущенные значения:")
df.isna().sum()

Пропущенные значения:


,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [70]:
# Статистические данные
print("Статистические данные:")
df.describe()

Статистические данные:


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### **Посчитайте процент выживаемости у каждого класса пассажиров (Pclass)**

In [71]:
# Только процент живых в каждом классе
result = df.groupby('Pclass')['Survived'].mean() * 100
result

,Survived
Pclass,
1,62.962963
2,47.282609
3,24.236253


In [72]:
# Добавим абсолютные значения
result = df.groupby('Pclass').agg(
    # Общее количество значений
    total_count=('Survived', 'size'),
    # Количество значений 1
    total_survived=('Survived', lambda x: (x == 1).sum()),
    # Количество значений 0
    total_died=('Survived', lambda x: (x == 0).sum()),
    # Процент значений 1
    survived_pct=('Survived', lambda x: (x.sum() / len(x)) * 100)
).reset_index()
result

,Pclass,total_count,total_survived,total_died,survived_pct
0,1,216,136,80,62.962963
1,2,184,87,97,47.282609
2,3,491,119,372,24.236253


### **Выведите самое популярное мужское и самое популярное женское имя на корабле**

In [73]:
# Подсчет количества каждой пары 'Sex' - 'first_name'
count_pairs = df.value_counts(subset=['Sex', 'first_name']).reset_index(name='Количество')
count_pairs[:4]

,Sex,first_name,Количество
0,male,William,35
1,male,John,23
2,female,Anna,14
3,male,George,13


In [74]:
# Вычисляем наиболее часто встречающуюся пары 'Sex' - 'first_name'
most_common = df.groupby('Sex')['first_name'].agg(lambda x: x.value_counts().idxmax()).reset_index()
most_common

,Sex,first_name
0,female,Anna
1,male,William


### **Выведите самое популярное мужское и самое популярное женское имя на корабле в каждом классе**

In [75]:
# Подсчет количества каждой 'Sex' - 'Pclass' - 'first_name'
cnt = df.value_counts(subset=['Sex', 'Pclass', 'first_name']).reset_index(name='Количество')
cnt[:4]

,Sex,Pclass,first_name,Количество
0,male,3,William,15
1,male,1,William,11
2,male,3,John,10
3,male,2,William,9


In [76]:
# Вычисляем наиболее часто встречающуюся 'Sex' - 'Pclass' - 'first_name'
most_common = df.groupby(['Sex', 'Pclass'])['first_name'].agg(lambda x: x.value_counts().idxmax()).reset_index()
most_common

,Sex,Pclass,first_name
0,female,1,Elizabeth
1,female,2,Elizabeth
2,female,3,Anna
3,male,1,William
4,male,2,William
5,male,3,William


### **Выведите часть таблицы с пассажирами, возраст которых больше 44 лет**

In [77]:
age_above_44 = df[df['Age'] > 44]
age_above_44[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,first_name
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Timothy
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,Elizabeth
15,16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,NaN,S,Mary
33,34,0,2,"Wheadon, Mr. Edward H",male,66.0,0,0,C.A. 24579,10.5000,NaN,S,Edward


### **Выведите часть таблицы с пассажирами, возраст которых меньше 44 лет и которые мужского пола**

In [78]:
age_below_44_male = df[(df['Age'] < 44) & (df['Sex']=='male')]
age_below_44_male[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,first_name
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.250,NaN,S,Owen
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.050,NaN,S,William
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,NaN,S,Gosta
12,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.050,NaN,S,William


### **Выведите количества n-местных кабин (в которых было 2, 3, 4, ... человека)**

In [83]:
df.groupby('Cabin').size().value_counts()

,count
1,101
2,38
3,5
4,3
